# Hyperparameter optimization

## Import packages

In [1]:
from rdkit import RDLogger
import logging
import warnings

warnings.filterwarnings("ignore")
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
RDLogger.DisableLog('rdApp.*')

from sklearn.metrics import accuracy_score
from deepmol.metrics import Metric
from sklearn.svm import SVC
from deepmol.parameter_optimization import HyperparameterOptimizerValidation

from deepmol.splitters import RandomSplitter
from deepmol.compound_featurization import MorganFingerprint
from deepmol.loaders import SDFLoader


from tensorflow.keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers

2023-06-29 13:47:44.314433: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 13:47:44.343799: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-29 13:47:44.344425: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-29 13:47:44.885955: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## First of all, let's load the data

In [2]:
dataset = SDFLoader("../data/CHEMBL217_conformers.sdf", id_field="_ID", labels_fields=["_Class"]).create_dataset()
train_dataset, valid_dataset, test_dataset = RandomSplitter().train_valid_test_split(dataset, frac_train=0.8, frac_valid=0.1, frac_test=0.1)

2023-06-29 13:48:10,595 — INFO — Assuming classification since there are less than 10 unique y values. If otherwise, explicitly set the mode to 'regression'!


## Let's featurize the data

In [3]:
morgan_fingerprints = MorganFingerprint()
morgan_fingerprints.featurize(train_dataset, inplace=True)
morgan_fingerprints.featurize(valid_dataset, inplace=True)
morgan_fingerprints.featurize(test_dataset, inplace=True)

## Hyperparameter tuning with scikit-learn

DeepMol provide methods to perform hyperparameter tuning with scikit-learn models. The hyperparameter tuning can be performed with a validation set, previously created, or with cross validation. Anyway, the hyperparameter tuning is performed with a random search if the number of iterations is specified, otherwise a grid search is performed.

Moreover, for each method the user have to specify the metric to optimize and if the metric has to be maximized or minimized. The user must specify the parameters to optimize and the values to try for each parameter.

The parameters must be specified as a dictionary, where the keys are the parameters names and the values are the values to try.

Let's see how to perform hyperparameter tuning of a SVM with a validation set.

In [4]:
params_dict_svc = {"C": [1.0, 1.2, 0.8], "kernel": ['linear', 'poly', 'rbf']} # The keys are the parameters names and the values are the values to try
optimizer = HyperparameterOptimizerValidation(SVC,
                                              metric=Metric(accuracy_score),
                                              maximize_metric=True,
                                              n_iter_search=2,
                                              params_dict=params_dict_svc,
                                              model_type="sklearn")
best_svm, best_hyperparams, all_results = optimizer.fit(train_dataset=train_dataset, valid_dataset=valid_dataset)

2023-06-29 13:48:33,584 — INFO — Fitting 2 random models from a space of 9 possible models.
2023-06-29 13:48:33,585 — INFO — Fitting model 1/2
2023-06-29 13:48:33,586 — INFO — hyperparameters: {'C': 1.0, 'kernel': 'rbf'}
2023-06-29 13:49:09,693 — INFO — Model 1/2, Metric accuracy_score, Validation set 1: 0.983755
2023-06-29 13:49:09,693 — INFO — 	best_validation_score so far: 0.983755
2023-06-29 13:49:09,694 — INFO — Fitting model 2/2
2023-06-29 13:49:09,694 — INFO — hyperparameters: {'C': 0.8, 'kernel': 'rbf'}
2023-06-29 13:49:46,486 — INFO — Model 2/2, Metric accuracy_score, Validation set 2: 0.982551
2023-06-29 13:49:46,486 — INFO — 	best_validation_score so far: 0.983755
2023-06-29 13:50:27,809 — INFO — Best hyperparameters: {'C': 1.0, 'kernel': 'rbf'}
2023-06-29 13:50:27,810 — INFO — train_score: 0.994811
2023-06-29 13:50:27,810 — INFO — validation_score: 0.983755


In the end, we can check the performance of the best model on the test set.

In [5]:
best_svm.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9873722188815394}, {})

We can also check the best combination of hyperparameters found.

In [6]:
best_hyperparams

{'C': 1.0, 'kernel': 'rbf'}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [7]:
all_results

{'_C_1.000000_kernel_rbf': 0.983754512635379,
 '_C_0.800000_kernel_rbf': 0.9825511432009627}

Finally, save your best model for deployment and new predictions!

In [8]:
best_svm.save("my_model")

Bring it back to life and make predictions!

In [9]:
from deepmol.models import SklearnModel

SklearnModel.load("my_model").predict(test_dataset)

array([0., 0., 0., ..., 1., 0., 0.])

## Now let's try with cross validation

In [11]:
from deepmol.parameter_optimization import HyperparameterOptimizerCV

params_dict_svc = {"C": [1.0, 1.2, 0.8], "kernel": ['linear', 'poly', 'rbf']}
optimizer = HyperparameterOptimizerCV(SVC, metric=Metric(accuracy_score),
                                          maximize_metric=True,
                                          cv=3,
                                          n_iter_search=2,
                                          params_dict=params_dict_svc,
                                          model_type="sklearn")
best_svm, best_hyperparams, all_results = optimizer.fit(train_dataset=train_dataset)

2023-06-29 17:13:36,013 — INFO — MODEL TYPE: sklearn
2023-06-29 17:13:36,014 — INFO — Fitting 2 random models from a space of 9 possible models.
2023-06-29 17:16:14,964 — INFO — 
 
 Best <function accuracy_score at 0x7f55eba87b80>: 0.981200 using {'kernel': 'poly', 'C': 0.8}
2023-06-29 17:16:14,964 — INFO — 
 <function accuracy_score at 0x7f55eba87b80>: 0.981200 (0.000907) with: {'kernel': 'poly', 'C': 0.8} 

2023-06-29 17:16:14,965 — INFO — 
 <function accuracy_score at 0x7f55eba87b80>: 0.972627 (0.001673) with: {'kernel': 'linear', 'C': 1.0} 

2023-06-29 17:16:14,966 — INFO — Fitting best model!
2023-06-29 17:16:53,755 — INFO — SklearnModel(mode='classification', model=SVC(C=0.8, kernel='poly'),
             model_dir='/tmp/tmpybobff7g')


Then, we can check the performance of the best model on the test set.

In [12]:
best_svm.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9849669272399278}, {})

We can also check the best combination of hyperparameters found.

In [13]:
best_hyperparams

{'kernel': 'poly', 'C': 0.8}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [17]:
all_results

{'mean_fit_time': array([20.26896993,  7.89151009]),
 'std_fit_time': array([0.27499022, 0.29743476]),
 'mean_score_time': array([9.4404827 , 2.50866826]),
 'std_score_time': array([0.2355606 , 0.10412309]),
 'param_kernel': masked_array(data=['poly', 'linear'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[0.8, 1.0],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'poly', 'C': 0.8}, {'kernel': 'linear', 'C': 1.0}],
 'split0_test_score': array([0.9799233 , 0.97496052]),
 'split1_test_score': array([0.98172795, 0.97180239]),
 'split2_test_score': array([0.98194946, 0.97111913]),
 'mean_test_score': array([0.98120024, 0.97262735]),
 'std_test_score': array([0.00090745, 0.00167322]),
 'rank_test_score': array([1, 2], dtype=int32)}

Finally, save your best model for deployment and new predictions!

In [18]:
best_svm.save("my_model")

Bring it back to life and make predictions!

In [19]:
from deepmol.models import SklearnModel

SklearnModel.load("my_model").predict(test_dataset)

array([0., 0., 0., ..., 1., 0., 0.])

# Hyperparameter tuning with keras

DeepMol provide methods to perform hyperparameter tuning with keras models. The hyperparameter tuning can be performed with a validation set, previously created, or with cross validation. Anyway, the hyperparameter tuning is performed with a random search if the number of iterations is specified, otherwise a grid search is performed.

As explained in the models section, to create a Keras model one have to define a function with the model architecture and the parameters to optimize.

In [20]:
def create_model(input_dim, optimizer='adam', dropout=0.5):
    # create model
    inputs = layers.Input(shape=input_dim)

    # Define the shared layers
    shared_layer_1 = layers.Dense(64, activation="relu")
    dropout_1 = Dropout(dropout)
    shared_layer_2 = layers.Dense(32, activation="relu")

    # Define the shared layers for the inputs
    x = shared_layer_1(inputs)
    x = dropout_1(x)
    x = shared_layer_2(x)

    task_output = layers.Dense(1, activation="sigmoid")(x)

    # Define the model that outputs the predictions for each task
    model = keras.Model(inputs=inputs, outputs=task_output)
    # Compile the model with different loss functions and metrics for each task
    model.compile(
        optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"]
    )
    return model


Let's see how to perform hyperparameter tuning of a DNN with a validation set.

In [21]:
params_dict_dense = {
                   "input_dim": [train_dataset.X.shape[1]],
                   "dropout": [0.5, 0.6, 0.7],
                   "optimizer": ["adam"]
                   }

optimizer = HyperparameterOptimizerValidation(create_model,
                                              metric=Metric(accuracy_score),
                                              maximize_metric=True,
                                              n_iter_search=2,
                                              params_dict=params_dict_dense,
                                              model_type="keras")


best_dnn, best_hyperparams, all_results = optimizer.fit(train_dataset=train_dataset, valid_dataset=valid_dataset)

2023-06-29 17:18:47,645 — INFO — Fitting 2 random models from a space of 3 possible models.
2023-06-29 17:18:47,646 — INFO — Fitting model 1/2
2023-06-29 17:18:47,647 — INFO — hyperparameters: {'input_dim': 2048, 'dropout': 0.5, 'optimizer': 'adam'}


2023-06-29 17:18:47.688410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-29 17:18:47.732633: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108937216 exceeds 10% of free system memory.


52/52 [==============================] - 0s 589us/step
2023-06-29 17:21:47,008 — INFO — Model 1/2, Metric accuracy_score, Validation set 1: 0.980144
2023-06-29 17:21:47,008 — INFO — 	best_validation_score so far: 0.980144
2023-06-29 17:21:47,009 — INFO — Fitting model 2/2
2023-06-29 17:21:47,009 — INFO — hyperparameters: {'input_dim': 2048, 'dropout': 0.7, 'optimizer': 'adam'}


2023-06-29 17:21:47.047236: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108937216 exceeds 10% of free system memory.


52/52 [==============================] - 0s 579us/step
2023-06-29 17:24:49,640 — INFO — Model 2/2, Metric accuracy_score, Validation set 2: 0.977738
2023-06-29 17:24:49,640 — INFO — 	best_validation_score so far: 0.980144
270/416 [==================>...........] - ETA: 0s

2023-06-29 17:24:49.657093: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108937216 exceeds 10% of free system memory.


416/416 [==============================] - 0s 567us/step
2023-06-29 17:24:50,068 — INFO — Best hyperparameters: {'input_dim': 2048, 'dropout': 0.5, 'optimizer': 'adam'}
2023-06-29 17:24:50,068 — INFO — train_score: 0.999925
2023-06-29 17:24:50,069 — INFO — validation_score: 0.980144


In the end, we can check the performance of the best model on the test set.

In [22]:
best_dnn.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

52/52 [==============================] - 0s 586us/step


({'accuracy_score': 0.9825616355983163}, {})

We can also check the best combination of hyperparameters found.

In [23]:
best_hyperparams

{'input_dim': 2048, 'dropout': 0.5, 'optimizer': 'adam'}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [25]:
all_results

{'_dropout_0.500000_input_dim_2048_optimizer_adam': 0.98014440433213,
 '_dropout_0.700000_input_dim_2048_optimizer_adam': 0.9777376654632972}

In [27]:
optimizer = HyperparameterOptimizerCV(create_model,
                                      metric=Metric(accuracy_score),
                                      maximize_metric=True,
                                      cv=3,
                                      n_iter_search=2,
                                      params_dict=params_dict_dense,
                                      model_type="keras")
params_dict_dense = {
                   "input_dim": [train_dataset.X.shape[1]],
                   "dropout": [0.5, 0.6, 0.7],
                   "optimizer": ["adam"]
                   }
best_dnn, best_hyperparams, all_results = optimizer.fit(train_dataset=train_dataset)

2023-06-29 17:27:56,111 — INFO — MODEL TYPE: keras
2023-06-29 17:27:56,112 — INFO — Fitting 2 random models from a space of 3 possible models.
139/139 [==============================] - 0s 566us/step


2023-06-29 17:28:02.068092: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108937216 exceeds 10% of free system memory.


416/416 [==============================] - 1s 1ms/step - loss: 0.1524 - accuracy: 0.9462
2023-06-29 17:28:02,977 — INFO — 
 
 Best <function accuracy_score at 0x7f55eba87b80>: 0.974808 using {'optimizer': 'adam', 'input_dim': 2048, 'dropout': 0.5}
2023-06-29 17:28:02,978 — INFO — 
 <function accuracy_score at 0x7f55eba87b80>: 0.974808 (0.001972) with: {'optimizer': 'adam', 'input_dim': 2048, 'dropout': 0.5} 

2023-06-29 17:28:02,978 — INFO — 
 <function accuracy_score at 0x7f55eba87b80>: 0.973605 (0.000641) with: {'optimizer': 'adam', 'input_dim': 2048, 'dropout': 0.6} 

2023-06-29 17:28:02,979 — INFO — Fitting best model!


2023-06-29 17:28:03.016279: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 108937216 exceeds 10% of free system memory.


Finally, save your best model for deployment and new predictions!

In [29]:
best_dnn.save("my_model")

Bring it back to life and make predictions!

In [30]:
from deepmol.models import KerasModel

best_dnn = KerasModel.load("my_model")

In [31]:
best_dnn.predict(test_dataset)

52/52 [==============================] - 0s 589us/step


array([0, 0, 0, ..., 1, 0, 0])

# Hyperparameter tuning with deepchem models

DeepMol provide methods to perform hyperparameter tuning with deepchem models. The hyperparameter tuning can be performed with a validation set, previously created, or with cross validation. Anyway, the hyperparameter tuning is performed with a random search if the number of iterations is specified, otherwise a grid search is performed.

As explained in the models section, to create a DeepChem model one have to define a function with the model architecture and the parameters to optimize. It is MANDATORY to pass the **model_type** parameter to the model.

Let's see how to perform hyperparameter tuning of a GraphConvModel with a validation set.

First, we have to featurize the dataset. These features are specific for the model we want to train. In this case, we want to train a GraphConvModel, so we have to featurize the dataset with ConvMolFeat.
For more documentation on this matter check the [DeepChem documentation](https://deepchem.readthedocs.io/en/latest/api_reference/featurizers.html).

In [32]:
from deepmol.compound_featurization import ConvMolFeat

ConvMolFeat().featurize(train_dataset, inplace=True)
ConvMolFeat().featurize(valid_dataset, inplace=True)
ConvMolFeat().featurize(test_dataset, inplace=True)

In [33]:
from deepmol.parameter_optimization import HyperparameterOptimizerValidation
from deepmol.parameter_optimization import HyperparameterOptimizerCV

from sklearn.metrics import roc_auc_score, precision_score
from deepmol.models import DeepChemModel
from deepchem.models import GraphConvModel

def graphconv_builder(graph_conv_layers, batch_size=256, epochs=5):
    graph = GraphConvModel(n_tasks=1, graph_conv_layers=graph_conv_layers, batch_size=batch_size,
                           mode='classification')
    return DeepChemModel(graph, epochs=epochs)

model_graph = HyperparameterOptimizerValidation(model_builder=graphconv_builder,
                                                metric=Metric(accuracy_score),
                                                maximize_metric=True,
                                                n_iter_search=2,
                                                params_dict={'graph_conv_layers': [[64, 64], [32, 32]]},
                                                model_type="deepchem")

best_model, best_hyperparams, all_results = model_graph.fit(train_dataset=train_dataset,valid_dataset=valid_dataset)

2023-06-29 17:32:49,453 — INFO — Fitting 2 random models from a space of 2 possible models.
2023-06-29 17:32:49,454 — INFO — Fitting model 1/2
2023-06-29 17:32:49,455 — INFO — hyperparameters: {'graph_conv_layers': [64, 64]}
2023-06-29 17:33:14,791 — INFO — Model 1/2, Metric accuracy_score, Validation set 1: 0.951264
2023-06-29 17:33:14,792 — INFO — 	best_validation_score so far: 0.951264
2023-06-29 17:33:14,792 — INFO — Fitting model 2/2
2023-06-29 17:33:14,792 — INFO — hyperparameters: {'graph_conv_layers': [32, 32]}
2023-06-29 17:33:36,465 — INFO — Model 2/2, Metric accuracy_score, Validation set 2: 0.943442
2023-06-29 17:33:36,466 — INFO — 	best_validation_score so far: 0.951264
2023-06-29 17:33:37,373 — INFO — Best hyperparameters: {'graph_conv_layers': [64, 64]}
2023-06-29 17:33:37,373 — INFO — train_score: 0.955106
2023-06-29 17:33:37,374 — INFO — validation_score: 0.951264


In the end, we can check the performance of the best model on the test set.

In [6]:
best_model.evaluate(test_dataset, metrics = [Metric(accuracy_score)])

({'accuracy_score': 0.9428743235117258}, {})

We can also check the best combination of hyperparameters found.

In [7]:
best_hyperparams

{'graph_conv_layers': [64, 64]}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [8]:
all_results

{'_graph_conv_layers_[64, 64]': 0.9494584837545126,
 '_graph_conv_layers_[32, 32]': 0.9199759326113117}

Finally, save your best model for deployment and new predictions!

In [9]:
best_model.save("my_model")

Bring it back to life and make predictions!

In [10]:
best_model = DeepChemModel.load("my_model")

In [11]:
best_model.predict(test_dataset)

array([[0.06773414, 0.9322658 ],
       [0.08101802, 0.9189819 ],
       [0.10755827, 0.8924417 ],
       ...,
       [0.00282426, 0.9971757 ],
       [0.00113406, 0.99886596],
       [0.98225605, 0.01774395]], dtype=float32)

#### Now, let's try hyperparameter tuning with deepchem models and cross validation

In [ ]:
def graphconv_builder(graph_conv_layers, batch_size=256, epochs=5):
    graph = GraphConvModel(n_tasks=1, graph_conv_layers=graph_conv_layers, batch_size=batch_size,
                           mode='classification')
    return DeepChemModel(graph, epochs=epochs)

model_graph = HyperparameterOptimizerCV(model_builder=graphconv_builder,
                                        metric=Metric(roc_auc_score),
                                        n_iter_search=2,
                                        maximize_metric=True,
                                        cv = 2,
                                        params_dict={'graph_conv_layers': [[64, 64], [32, 32]]},
                                        model_type="deepchem")

best_model, best_hyperparameters, all_results = model_graph.fit(train_dataset=train_dataset)


In the end, we can check the performance of the best model on the test set.

In [13]:
test_preds = best_model.predict(test_dataset)

metrics = [Metric(roc_auc_score), Metric(precision_score), Metric(accuracy_score)]

best_model.evaluate(test_dataset, metrics)


({'roc_auc_score': 0.9905489702285326,
  'precision_score': 0.9622166246851386,
  'accuracy_score': 0.9542994588093806},
 {})

We can also check the best combination of hyperparameters found.

In [14]:
best_hyperparameters

{'graph_conv_layers': [64, 64]}

We can also check the performance of all the models trained during the hyperparameter tuning. Each model is defined by the name of the parameters followed by the value of the parameter.

In [21]:
all_results

defaultdict(list,
            {'params': [{'graph_conv_layers': [64, 64]},
              {'graph_conv_layers': [32, 32]}],
             'mean_train_score': [0.9795196072718013, 0.980248484393072],
             'mean_test_score': [0.9758974414293784, 0.9754465835745805],
             'std_train_score': [0.003408778455179895, 0.000607063345097747],
             'std_test_score': [0.0029443151372258725, 0.0011259723129122823],
             'split0_train_score': [0.9829283857269812, 0.9796414210479742],
             'split0_test_score': [0.9788417565666043, 0.9743206112616681],
             'split1_train_score': [0.9761108288166214, 0.9808555477381697],
             'split1_test_score': [0.9729531262921526, 0.9765725558874927]})

In [22]:
all_results["params"]

[{'graph_conv_layers': [64, 64]}, {'graph_conv_layers': [32, 32]}]

Finally, save your best model for deployment and new predictions!

In [16]:
best_model.save("my_model")

Bring it back to life and make predictions!

In [17]:
best_model = DeepChemModel.load("my_model")

In [18]:
best_model.predict(test_dataset)

array([[0.04331122, 0.95668876],
       [0.7563593 , 0.24364069],
       [0.00733165, 0.9926683 ],
       ...,
       [0.0438629 , 0.9561371 ],
       [0.06368961, 0.9363104 ],
       [0.9964748 , 0.00352522]], dtype=float32)